# Web Scrapping: Superintendencia de Salud

El siguiente código realiza una revisión y extracción del código fuente de la página de la Superintendencia de Salud. Los requisitos para poder ejecutar este código son:

 - **Instalar librerías:**
  - pandas
  - scrapy
  - requests
  - datetime
  - wget
 - **Tener creadas las carpetas para almacenar los archivos.**
  - En los **códigos 32 al 34** están las rutas en donde se almacenan, las cuales deben ser cambiadas por quien ejecute el código para que referencien a su propio equipo.
  
---
 - **Fecha de creación: 25.09.2020 (IIC)** 
 - **Fecha modificación_1: 01.10.2020 (IIC)**

## Importar librerías

In [1]:
import pandas as pd
from scrapy import Selector
import requests
from datetime import datetime, timedelta

## Cargar códigos html de las páginas a utilizar

In [2]:
html_circ_interna = requests.get('http://www.supersalud.gob.cl/normativa/668/w3-propertyvalue-6258.html').content
html_resoluciones = requests.get('http://www.supersalud.gob.cl/normativa/668/w3-propertyvalue-6259.html').content
html_oficio_circ = requests.get('http://www.supersalud.gob.cl/normativa/668/w3-propertyvalue-6260.html').content

### Crear selectors para después buscar la información

In [3]:
sel_ci = Selector(text = html_circ_interna)
sel_r = Selector(text = html_resoluciones)
sel_oc = Selector(text = html_oficio_circ)

## Búsqueda de información

### Lista de fechas

En la siguiente línea de código se busca el tag "td" que contenga como clase "fecha-publicacion"  en todo el código y se extrae el texto asociado.

In [4]:
fechas_ci = sel_ci.xpath('//td[contains(@class,"fecha-publicacion")]/text()').extract()
fechas_r = sel_r.xpath('//td[contains(@class,"fecha-publicacion")]/text()').extract()
fechas_oc = sel_oc.xpath('//td[contains(@class,"fecha-publicacion")]/text()').extract()

### Lista Títulos

En la siguiente línea de código se busca en todos los tag td que tengan como "hijo" un tag span con la clase "td_30_porciento titulo-principal" y este a su vez, tenga asociado un hipervículo del cual se extrae el título que tiene asociado.

In [5]:
titulos_ci = sel_ci.xpath('//td/span[contains(@class, "td_30_porciento titulo-principal")]/a/text()').extract()
titulos_r = sel_r.xpath('//td/span[contains(@class, "td_30_porciento titulo-principal")]/a/text()').extract()
titulos_oc = sel_oc.xpath('//td/span[contains(@class, "td_30_porciento titulo-principal")]/a/text()').extract()

### Lista con descipción

En la siguiente línea de código se busca el tag "span" que contenga como clase "fecha-publicacion" en todo el código y se extrae el texto asociado.

In [6]:
descripcion_ci = sel_ci.xpath('//span[contains(@class,"td_30_porciento resumen-modificaciones")]/text()').extract()
descripcion_r = sel_r.xpath('//span[contains(@class,"td_30_porciento resumen-modificaciones")]/text()').extract()
descripcion_oc = sel_oc.xpath('//span[contains(@class,"td_30_porciento resumen-modificaciones")]/text()').extract()

### Lista con links

En la siguiente línea de código se busca en todos los tag td que tengan como "hijo" un tag span con la clase "td_20_porciento cid-19" y este a su vez, tenga asociado un hipervículo el cual se extrae.

In [7]:
links_ci = sel_ci.xpath('//td/span[contains(@class, "td_20_porciento cid-19")]/a/@href').extract()

In [8]:
url_circ_internas = []

for i in links_ci:
    url = 'http://www.supersalud.gob.cl/normativa/668/'+i
    url_circ_internas.append(url)

In [9]:
links_r = sel_r.xpath('//td/span[contains(@class, "td_20_porciento cid-19")]/a/@href').extract()

In [10]:
url_resoluciones = []

for i in links_r:
    url = 'http://www.supersalud.gob.cl/normativa/668/'+i
    url_resoluciones.append(url)

In [11]:
links_oc = sel_oc.xpath('//td/span[contains(@class, "td_20_porciento cid-19")]/a/@href').extract()

In [12]:
url_oficio_circ = []

for i in links_oc:
    url = 'http://www.supersalud.gob.cl/normativa/668/'+i
    url_oficio_circ.append(url)

## Almacenar información obtenida y agregar formatos

In [13]:
df_circ_internas = pd.DataFrame({'Fecha Publicación' : fechas_ci, 'Título' : titulos_ci, 'Descripción' : descripcion_ci, 'Link': url_circ_internas})

#Agregamos formato fecha a la columna "Fecha Publicación"
df_circ_internas['Fecha Publicación'] = pd.to_datetime(df_circ_internas['Fecha Publicación'])

#Quitamos "/" de los títulos y reemplazamos por " "
df_circ_internas['Título'] = df_circ_internas['Título'].str.replace('/',' ')

In [14]:
df_resoluciones = pd.DataFrame({'Fecha Publicación' : fechas_r, 'Título' : titulos_r, 'Descripción' : descripcion_r, 'Link': url_resoluciones})

#Agregamos formato fecha a la columna "Fecha Publicación"
df_resoluciones['Fecha Publicación'] = pd.to_datetime(df_resoluciones['Fecha Publicación'])

#Quitamos "/" de los títulos y reemplazamos por " "
df_resoluciones['Título'] = df_resoluciones['Título'].str.replace('/',' ')

In [15]:
df_oficio_circ = pd.DataFrame({'Fecha Publicación' : fechas_oc, 'Título' : titulos_oc, 'Descripción' : descripcion_oc, 'Link': url_oficio_circ})

#Agregamos formato fecha a la columna "Fecha Publicación"
df_oficio_circ['Fecha Publicación'] = pd.to_datetime(df_oficio_circ['Fecha Publicación'])

#Quitamos "/" de los títulos y reemplazamos por " "
df_oficio_circ['Título'] = df_oficio_circ['Título'].str.replace('/',' ')

## Filtrar novedades

In [16]:
novedad = datetime.now() - timedelta(days=30)

### Circulares internas

In [17]:
df_circ_internas_novedad = df_circ_internas.loc[(df_circ_internas['Fecha Publicación']>=novedad)]
df_circ_internas_novedad

,Fecha Publicación,Título,Descripción,Link


In [18]:
link_circ_internas_novedad = df_circ_internas_novedad['Link']
titulos_circ_internas_novedad = df_circ_internas_novedad['Título'].str.replace(' ','')

### Resoluciones

In [19]:
df_resoluciones_novedad = df_resoluciones.loc[(df_resoluciones['Fecha Publicación']>=novedad)]
df_resoluciones_novedad

,Fecha Publicación,Título,Descripción,Link
0,2020-09-17,Resolución Exenta IP N° 3709,Resolución que ordena inscribir en el Registro...,http://www.supersalud.gob.cl/normativa/668/art...
15,2020-09-06,Resolución Exenta IP N° 1956,Resolución que ordena mantener con el N°228 en...,http://www.supersalud.gob.cl/normativa/668/art...
16,2020-09-06,Resolución Exenta IP N° 1955,Resolución que ordena inscribir en el Registro...,http://www.supersalud.gob.cl/normativa/668/art...
17,2020-09-06,Resolución Exenta IP N° 1942,Resolución que ordena inscribir en el Registro...,http://www.supersalud.gob.cl/normativa/668/art...
31,2020-12-05,Resolución Exenta IP N° 1656,Resolución que ordena inscribir en el Registro...,http://www.supersalud.gob.cl/normativa/668/art...
32,2020-12-05,Resolución Exenta IP N° 1655,Resolución que ordena inscribir en el Registro...,http://www.supersalud.gob.cl/normativa/668/art...
55,2020-09-04,Resolución Exenta IP N° 1374,Resolución que ordena mantener con el N°225 en...,http://www.supersalud.gob.cl/normativa/668/art...
56,2020-09-04,Resolución Exenta IP N° 1371,Resolución que ordena inscribir en el Registro...,http://www.supersalud.gob.cl/normativa/668/art...
57,2020-09-04,Resolución Exenta IP N° 1367,Resolución que ordena inscribir en el Registro...,http://www.supersalud.gob.cl/normativa/668/art...
81,2020-12-03,Resolución Exenta IP N° 981,Resolución que ordena inscribir en el Registro...,http://www.supersalud.gob.cl/normativa/668/art...


In [20]:
link_resoluciones_novedad = df_resoluciones_novedad['Link']
titulos_resoluciones_novedad = df_resoluciones_novedad['Título'].str.replace(' ','')

### Oficios circulares

In [21]:
df_oficio_circ_novedad = df_oficio_circ.loc[(df_oficio_circ['Fecha Publicación']>=novedad)]
df_oficio_circ_novedad

,Fecha Publicación,Título,Descripción,Link
5,2020-10-03,Ordinario Circular IP N° 2,Instruye sobre el cumplimiento de la Ley N°20....,http://www.supersalud.gob.cl/normativa/668/art...


In [22]:
link_oficio_circ_novedad = df_oficio_circ_novedad['Link']

In [23]:
titulos_oficio_circ_novedad = df_oficio_circ_novedad['Título']

## Descargar novedades

### Rutas para almacenar la información

**Circulares Internas**

In [24]:
path_ci = []

for i in titulos_circ_internas_novedad:
    # La ruta escrita entre '' debe ser cambiada por quien ejecute el código.
    path = 'C:/projects/datalake/data/output/current/cambios_normativos/super_salud/Circulares_internas/'+i
    path_ci.append(path)

**Resoluciones**

In [25]:
path_r = []

for i in titulos_resoluciones_novedad:
    # La ruta escrita entre '' debe ser cambiada por quien ejecute el código.
    path = 'C:/projects/datalake/data/output/current/cambios_normativos/super_salud/Resoluciones/'+i
    path_r.append(path)

**Oficio Circular**

In [26]:
path_oc = []

for i in titulos_oficio_circ_novedad:
    # La ruta escrita entre '' debe ser cambiada por quien ejecute el código.
    path = 'C:/projects/datalake/data/output/current/cambios_normativos/super_salud/Oficio_Circular/'+i
    path_oc.append(path)

### Descargar archivos

In [27]:
import wget

**Circulares internas**

In [29]:
i = 0

for url in link_circ_internas_novedad:
    wget.download(url,path_ci[i]+'.pdf')
    i = i + 1

**Resoluciones**

In [30]:
i = 0

for url in link_resoluciones_novedad:
    wget.download(url,path_r[i]+'.pdf')
    i = i + 1

**Circulares internas**

In [31]:
i = 0

for url in link_oficio_circ_novedad:
    wget.download(url,path_oc[i]+'.pdf')
    i = i + 1

### Guardar DF históricos en csv

In [32]:
output_path = 'C:/projects/datalake/data/output/history/cambios_normativos/super_salud/'

df_circ_internas.to_csv(output_path + 'circulares_internas' + '.csv')
df_resoluciones.to_csv(output_path + 'resoluciones' + '.csv')
df_oficio_circ.to_csv(output_path + 'oficio_circular' + '.csv')